- READ IN FILE
- CLEAN TEXT
- BREAK INTO SENTENCES
- EXTRACT SUBJECT OBJECT
- EXTRACT RELATION
- PRINT RESULT
- PRODUCE GRAPH

In [2]:
# IMPORT
import spacy
# import scispacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import tqdm
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt
from spacy.lang.en import English # updated
nlp = spacy.load('en_core_web_sm')

In [3]:
# Readin to pandas Data Frame
candidate_sentences = pd.read_csv("/Users/awenc/NUIM/CS440/KG_NLPSystem/data/sentences_psychology.csv")
candidate_sentences.shape
candidate_sentences['sentences'].sample(5)

812    A major aquarium in a city on the East Coast d...
804    Thedesigner considered shining a powerful spot...
463    When the tunnel was dug, the men crawled throu...
585    When he ranout of money he paid a visit to his...
801    In order to maintainthe deep water environment...
Name: sentences, dtype: object

In [4]:
def clean(text):
    # removing new line character
    text = re.sub('\n','', str(text))
    text = re.sub('\n ','',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("- ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    text = re.sub("�s",'',str(text))

    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("- ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))

    return text

In [5]:
def sent_(sentence, dep=False):
    """
    Analyse sentence, breaks sentence into: text, pos, dep
    part of speach
    dependency

    """

    sent = nlp(sentence)
    for token in sent:
        # Get the token text, part-of-speech tag and dependency label
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        # This is for formatting only
        print('{:<12}{:<10}{:<10}{:<10}'.format(token_text, token_pos, token_dep,spacy.explain(token_pos)))
    if dep==True:
        displacy.render(sent, style='dep')
        # print("Displacy File:",displacy.__file__)

In [6]:
def sentencize(text):
    """
    Take in text and break it into sentences
    """
    nlp = English()
    nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
    doc = nlp(text)
    sentences = [sent.string.strip() for sent in doc.sents]
    # sentences = nlp(text)
    # for i, sentence in enumerate(sentences.sents):
    #     print(i, sentence)
    #     return sentence
    return sentences

In [7]:
def fileconvert():
    """
    Read in direcory of files, look for .txt extension, extract sentences from text, save sentences in one file,
    """
    from pathlib import Path
    p = Path('/Users/awenc/NUIM/CS440/KG_NLPSystem/data/Psychology Test Materials')
    for name in p.glob('*.txt'):
        f = open(name, 'r')
        line = f.read()
        # print(line)
        x = get_sent(line)
        len(get_sent(line))
        outfile = open("/Users/awenc/NUIM/CS440/KG_NLPSystem/workspace/sentences_psychology.txt",'a')
        type(x)
        for i in clean(x):
            # print(type(i)) # <class 'spacy.tokens.span.Span'>
            # print(str(i))
            outfile.write(str(i)+"\n")
    """
    Wrap each line in quotes in order to make data digastable by pandas DataFrame
    """
    with open("/Users/awenc/NUIM/CS440/KG_NLPSystem/workspace/sentences_psychology.txt",'r') as f:
        x= f.readlines()
        with open('/Users/awenc/NUIM/CS440/KG_NLPSystem/workspace/sentences_psychology.csv','w') as fw:
            fw.write("sentences"+"\n")
            for line in x:
                fw.write('\"'+line.strip('\n').strip('\r')+'\"\n')

In [50]:
def get_triple(text):

    doc = nlp(text)

    sent = []

    for token in doc:

        # if the token is a verb
        # if (token.pos_=='VERB') or (token.pos_=='ROOT'):
        if (token.pos_ in ['VERB','ROOT']):
            phrase =''

            # only extract noun or pronoun subjects
            for sub_tok in token.lefts:

                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):

                    # add subject to the phrase
                    phrase += sub_tok.text

                    # save the root of the verb in phrase
                    phrase += ' '+token.lemma_

                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:

                        # save the object in the phrase
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):

                            phrase += ' '+sub_tok.text
                            sent.append(phrase)

    return sent

In [36]:
txt = "Two fairly poor brothers, Ben and Jerry, had just inherited a working farm whose main crop has a volatile price. Ben wanted to sell rights to the farm�s output under a long-term contract for a fixed amount rather than depend upon shares of an uncertain revenue stream. In short, Ben was risk averse. Jerry, on the other hand, was confident that the next season would be spectacular and revenues would be high. In short, Jerry was risk seeking. The two argued for days and nights. Ben wanted to sell immediately because he believed the price of the crop would fall; Jerry wanted to keep the farm because he believed the price of the crop would increase. Jerry could not afford to buy Ben out at the time, but the strain on their family relationship over their disagreements was becoming too large. Ben had always trusted Jerry�s instincts in the past, but this time felt Jerry was being overly optimistic. In an effort to settle the matter and close up the growing rift between them, the two brothers agreed to meet with a family business advisor. Following the consultation, Jerry proposed a possible agreement to his brother: They would keep the farm for another season. If the price of the crop fell below a certain price (as Ben thought it would), then they would sell the farm and Ben would get 50% of the farm�s current value, adjusted for inflation; Jerry would get the rest. However, if the price of the crop were to rise (as Jerry thought it would), Jerry would buy Ben out for 50% of the farm�s current value, adjusted for inflation, and would get to keep all of the additional profits for himself. Jerry was delighted when his brother told him he could agree to this arrangement, thereby avoiding further conflict."

In [37]:
txt = clean(txt)

In [38]:
txt

'Two fairly poor brothers, Ben and Jerry, had just inherited a working farm whose main crop has a volatile price. Ben wanted to sell rights to the farm output under a long term contract for a fixed amount rather than depend upon shares of an uncertain revenue stream. In short, Ben was risk averse. Jerry, on the other hand, was confident that the next season would be spectacular and revenues would be high. In short, Jerry was risk seeking. The two argued for days and nights. Ben wanted to sell immediately because he believed the price of the crop would fall; Jerry wanted to keep the farm because he believed the price of the crop would increase. Jerry could not afford to buy Ben out at the time, but the strain on their family relationship over their disagreements was becoming too large. Ben had always trusted Jerry instincts in the past, but this time felt Jerry was being overly optimistic. In an effort to settle the matter and close up the growing rift between them, the two brothers agr

In [39]:
x1 = sent_(txt)
# x1 = nounverbnoun(txt)

Two         NUM       nummod    numeral   
fairly      ADV       advmod    adverb    
poor        ADJ       amod      adjective 
brothers    NOUN      nsubj     noun      
,           PUNCT     punct     punctuation
Ben         PROPN     appos     proper noun
and         CCONJ     cc        coordinating conjunction
Jerry       PROPN     conj      proper noun
,           PUNCT     punct     punctuation
had         AUX       aux       auxiliary 
just        ADV       advmod    adverb    
inherited   VERB      ROOT      verb      
a           DET       det       determiner
working     VERB      amod      verb      
farm        NOUN      dobj      noun      
whose       DET       poss      determiner
main        ADJ       amod      adjective 
crop        NOUN      nsubj     noun      
has         AUX       relcl     auxiliary 
a           DET       det       determiner
volatile    ADJ       amod      adjective 
price       NOUN      dobj      noun      
.           PUNCT     punct     punc

In [40]:
x1

In [42]:
get_relation(txt)

'was delighted'

In [43]:
ent_extraction(txt)

['he', 'further  conflict']

In [44]:
sr = sentencize(txt)

In [45]:
for i in sr:
    print(get_relation(i))
    print(ent_extraction(i))

inherited
['main  crop', 'volatile  price']
wanted
['Ben', 'uncertain uncertain revenue stream']
was
['short  Ben', '']
was confident
['revenues', 'other  hand']
was
['short  Jerry', '']
argued for
['two', 'days']
wanted
['price', 'crop']
wanted
['price', 'crop']
afford
['strain', 'their family disagreements']
trusted Jerry
['Jerry', 'always trusted Jerry past']
agreed
['two  brothers', 'family business advisor']
proposed
['Jerry', 'possible  brother']
keep
['They', 'season']
sell
['Ben', 'current  inflation']
get
['Jerry', 'rest']
buy Ben
['Jerry', 'additional  himself']
was delighted
['he', 'further  conflict']


In [51]:
for i in sr:
    print(sent_(i))
    print(nounverbnoun(i))

Two         NUM       nummod    numeral   
fairly      ADV       advmod    adverb    
poor        ADJ       amod      adjective 
brothers    NOUN      nsubj     noun      
,           PUNCT     punct     punctuation
Ben         PROPN     appos     proper noun
and         CCONJ     cc        coordinating conjunction
Jerry       PROPN     conj      proper noun
,           PUNCT     punct     punctuation
had         AUX       aux       auxiliary 
just        ADV       advmod    adverb    
inherited   VERB      ROOT      verb      
a           DET       det       determiner
working     VERB      amod      verb      
farm        NOUN      dobj      noun      
whose       DET       poss      determiner
main        ADJ       amod      adjective 
crop        NOUN      nsubj     noun      
has         AUX       relcl     auxiliary 
a           DET       det       determiner
volatile    ADJ       amod      adjective 
price       NOUN      dobj      noun      
.           PUNCT     punct     punc

In [ ]:
ent_pairs = []
relations = []
nvn = []
# relations = [get_relation(i) for i in tqdm(candidate_sentences['sentences'])]
for i in candidate_sentences["sentences"]:
    try:
        ent_pairs.append(ent_extraction(i))
        relations.append(get_relation(i))
        nvn.append(ent_extraction(i))
        nvn.append(get_relation(i))
    except TypeError as e:
        print(i)
        print(type(i))
        print(e)
        pass
kg_df = pd.DataFrame({'source':object_, 'target':subject_, 'edge':relations})

In [ ]:
kg_df
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [49]:
G=nx.from_pandas_edgelist(kg_df[kg_df['edge']=="was"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5)
nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

NameError: name 'kg_df' is not defined

In [88]:
# candidate_sentences['sentences'][0]
for i,j in zip(ent_pairs,relations):
    print(i[0],j,i[1])
# ent_pairs
# relations
# nvn

 who was rocky  cliff
a cross that saw feathers
bolts attacked 
her tail she realized her tail sportsman
sportsman was pleased 
He promised eagles
 Twins 
identical  who were identical continually  family
annual annual school picnic was big  twins
athletic  twins were prizes
who arrived 
twins wanted 2 2 mile school
secretly  which devised newcomer
day arrived race
where  teacher was winner
he entered broad broad jumping event
students began woods
others rushed into sight
Then  he went back the picnic area
who burst other  runners
teacher named running
year  twins switched thereafter  event
 Case 
Syd bought Asian wholesale leather manufacturer
aspects negotiated except successfully  boots
multinational  she decided aggressive  pricing
Asian  they told Syd boat
a trade embargo concerned near  future
the U.S. embargo told Syd the U.S. dock
multinational  conglomerate thought small  businesses
by air which wanted by air freight
Asian Asian sales representative refused because higher  cos

In [91]:
# object_ = [i[0] for i in ent_pairs]
# subject_ = [i[1] for i in ent_pairs]
# relations = [get_relation(i) for i in candidate_sentences['sentences']]
# kg_df = pd.DataFrame({'source':object_, 'target':subject_, 'edge':relations})

TypeError: object of type 'float' has no len()

In [52]:
def get_sents(text):
    """
    return:     list of sentences for given text,
                you can extract single sentence using
                indexing. It return str object.
    """
    tokens = nlp(text)
    sents = []
    for sent in tokens.sents:
        sents.append(sent.string.strip())
    # print(f"We got {len(sent)} sentences")
    return sents

In [56]:
get_sents(txt)

['Two fairly poor brothers, Ben and Jerry, had just inherited a working farm whose main crop has a volatile price.',
 'Ben wanted to sell rights to the farm output under a long term contract for a fixed amount rather than depend upon shares of an uncertain revenue stream.',
 'In short, Ben was risk averse.',
 'Jerry, on the other hand, was confident that the next season would be spectacular and revenues would be high.',
 'In short, Jerry was risk seeking.',
 'The two argued for days and nights.',
 'Ben wanted to sell immediately because he believed the price of the crop would fall; Jerry wanted to keep the farm because he believed the price of the crop would increase.',
 'Jerry could not afford to buy Ben out at the time, but the strain on their family relationship over their disagreements was becoming too large.',
 'Ben had always trusted Jerry instincts in the past, but this time felt Jerry was being overly optimistic.',
 'In an effort to settle the matter and close up the growing ri

In [55]:
sentencize(txt)

['Two fairly poor brothers, Ben and Jerry, had just inherited a working farm whose main crop has a volatile price.',
 'Ben wanted to sell rights to the farm output under a long term contract for a fixed amount rather than depend upon shares of an uncertain revenue stream.',
 'In short, Ben was risk averse.',
 'Jerry, on the other hand, was confident that the next season would be spectacular and revenues would be high.',
 'In short, Jerry was risk seeking.',
 'The two argued for days and nights.',
 'Ben wanted to sell immediately because he believed the price of the crop would fall;',
 'Jerry wanted to keep the farm because he believed the price of the crop would increase.',
 'Jerry could not afford to buy Ben out at the time, but the strain on their family relationship over their disagreements was becoming too large.',
 'Ben had always trusted Jerry instincts in the past, but this time felt Jerry was being overly optimistic.',
 'In an effort to settle the matter and close up the growin